In [ ]:
# Changing the working directory
import os
os.chdir('../')

import gspread
from utils import *
from skydb.sheets import updateSpreadsheet

In [ ]:
records = getOfficialRecords()
grades = records['grades']
gpas = records['gpas']
students = getStudents()
psat_scores = getPsatScores()

In [ ]:
""" PSAT Scores """
math_psat = psat_scores[['user_id', 'math_score']]
psat_students = (
    students
    .query("""grade_level in [9,10,11]""")
    .drop('counselor', axis=1)
    .rename(columns={'id':'user_id'})
    .astype({'user_id':'int64', 'student_id':'float', 'grade_level':'int64'})
    .merge(math_psat,
                'left',
                on='user_id'
    )
)

In [ ]:
""" Algebra 1/2 Grades """
raw_math_grades = grades[grades.course_title.str.contains('Algebra \\d')].copy()
for index, row in raw_math_grades.iterrows():
    if row['school_year'] == '2021 - 2022' and row['term'] == 'Fall':
        raw_math_grades.loc[index, 'official_grade'] = row['grade']
    elif row['term'] == 'Spring':
        raw_math_grades.loc[index, 'official_grade'] = row['grade']
    else:
        raw_math_grades.loc[index, 'official_grade'] = None
        
math_grades = raw_math_grades[~raw_math_grades.official_grade.isna()].copy().drop('grade', axis=1)

raw_math_students = (
    psat_students
    .merge(math_grades,
           'inner',
           on='user_id'
    )
)

""" Label Alegebra Classes"""
for index, row in raw_math_students.iterrows():
    raw_math_students.loc[index, 'label'] = 'Algebra 1' if 'Algebra 1' in row['course_title'] else 'Algebra 2'
    
""" Spread Data """
clean_math_students = (
    raw_math_students
    .groupby(['user_id','label'])
    .official_grade
    .first()
    .unstack()
)

""" Joining with student Data """
math_students = (
    psat_students
    .merge(clean_math_students,
           'inner',
           on='user_id'
    )
)

In [ ]:
math_students

In [ ]:
""" Cumulative GPA """
math_data = (
    math_students
    .merge(gpas,
           'inner',
           on='user_id'
    )
    .sort_values(['last_name', 'first_name'])
    .drop(['user_id', 'student_id', 'first_name'], axis=1)
)

In [ ]:
math_data

In [ ]:
updateSpreadsheet(math_data.fillna(''), 
                  sheet_id='1xgA6DrPoU3QHGBMZVichqyikiA3X82C8yiUHn1QnxFE',
                  styleClass=HysonFireStyle
                 )

In [ ]:
# gc = gspread.service_account()
# spreadsheet = gc.open_by_key('1xgA6DrPoU3QHGBMZVichqyikiA3X82C8yiUHn1QnxFE')
# emails = [
#     'ebaker@tampaprep.org', 'sbeekman@tampaprep.org', 'pembry@tampaprep.org', 'jkeller@tampaprep.org',
#     'bsarkozy@tampaprep.org', 'rharwell@tampaprep.org', 
# ]
# for email in emails:
#     spreadsheet.share(email, 'user', 'reader')